In [1]:
import pyarrow.parquet as pq
import pandas as pd
import glob
import json
from pathlib import Path
import os

# creat df_ppo_var

In [2]:
home = str(Path.home())
path = os.path.join(home,'Téléchargements', 'mario_learning-sourcedata', 'sourcedata', 'ppo_*', 'sub-*', 'ses-*', 'beh', 'variables', '*.json')
file_list_base = glob.glob(path)

columns = ( 'Subject', 'World', 'Level', 'Scene','LevelFullName', 'SceneFullName','ClipCode',
        'Learning_Phase', 'player_x_posHi', 'player_x_posLo', 'player_y_pos','json_path')

df_ppo_var = pd.DataFrame(columns=columns)

df_ppo_var['json_path'] = file_list_base
df_ppo_var['Subject'] = 'ppo'
df_ppo_var[['World', 'Level', 'Scene', 'ClipCode']] = df_ppo_var['json_path'].str.extract(r'level-w(\d+)l(\d+)_scene-(\d+)_clip-(\d+).json')
df_ppo_var['LevelFullName'] = 'w' + df_ppo_var['World'] + 'l' + df_ppo_var['Level']
df_ppo_var['SceneFullName'] = df_ppo_var['World']+'-'+df_ppo_var['Level']+'-'+df_ppo_var['Scene']
df_ppo_var['Learning_Phase'] = df_ppo_var['json_path'].str.extract(r'ppo_mario_(ep-\w+)')

player_x_hi = []
player_x_lo = []
player_y = []

for path in df_ppo_var['json_path']:
    with open(path, "r") as f:
        data = json.load(f)

    player_x_hi.append(data["player_x_posHi"])
    player_x_lo.append(data["player_x_posLo"])
    player_y.append(data["player_y_pos"])

df_ppo_var['player_x_posHi'] = player_x_hi
df_ppo_var['player_x_posLo'] = player_x_lo
df_ppo_var['player_y_pos'] = player_y

df_ppo_var.to_parquet("../sourcedata/df_variables_ppo.parquet", index=False)

In [3]:
home = str(Path.home())
path = os.path.join(home,'github', 'mario.scenes','outputdata', 'outputdata','scene_clips', 'sub-*', 'ses-*', 'beh', 'variables', '*.json')
file_list_base = glob.glob(path)
#https://file+.vscode-resource.vscode-cdn.net/home/hugo/github/mario.scenes/outputdata/outputdata/scene_clips/sub-01/ses-004/beh/variables/sub-01_ses-004_run-03_level-w6l1_scene-5_clip-00403040000625.json
#'/home/hugo/gitlab.pavlovia/online_test/creat_sourcedata/select_scenes/sourcedata/clips_metadata_with_patterns.parquet' 

columns = ( 'Subject', 'World', 'Level', 'Scene','LevelFullName', 'SceneFullName','ClipCode',
        'Learning_Phase', 'player_x_posHi', 'player_x_posLo', 'player_y_pos','json_path')

# to add 'Phase', 'Phase_stage'

df_var_hum = pd.DataFrame(columns=columns)
df_var_hum['json_path'] = file_list_base
df_var_hum['Subject'] = df_var_hum['json_path'].str.extract(r'(sub-\d+)_ses')
df_var_hum[['World', 'Level', 'Scene', 'ClipCode']] = df_var_hum['json_path'].str.extract(r'level-w(\d+)l(\d+)_scene-(\d+)_clip-(\d+).json')
df_var_hum['ClipCode'] = df_var_hum['ClipCode'].astype(int)
df_var_hum['LevelFullName'] = 'w' + df_var_hum['World'] + 'l' + df_var_hum['Level']
df_var_hum['SceneFullName'] = df_var_hum['World']+'-'+df_var_hum['Level']+'-'+df_var_hum['Scene']

df_meta = pd.read_parquet(os.path.join(home,'gitlab.pavlovia', 'online_test','creat_sourcedata','select_scenes', 'sourcedata', 'clips_metadata_with_patterns.parquet'))
human_meta = df_meta[df_meta['Model'] == 'human']
human_meta = human_meta[['Subject', 'ClipCode', 'Learning_Phase']]
df_var_hum = df_var_hum.drop(columns=['Learning_Phase'])
df_var_hum['ClipCode'] = df_var_hum['ClipCode'].astype(int)
human_meta['ClipCode'] = human_meta['ClipCode'].astype(int)
df_var_hum = pd.merge(df_var_hum, human_meta, on=['Subject', 'ClipCode'], how='left')
player_x_hi = []
player_x_lo = []
player_y = []

for path in df_var_hum['json_path']:
    with open(path, "r") as f:
        data = json.load(f)

    player_x_hi.append(data["player_x_posHi"])
    player_x_lo.append(data["player_x_posLo"])
    player_y.append(data["player_y_pos"])

df_var_hum['player_x_posHi'] = player_x_hi
df_var_hum['player_x_posLo'] = player_x_lo
df_var_hum['player_y_pos'] = player_y

df_var_hum.to_parquet("../sourcedata/df_variables_hum.parquet", index=False)


In [ ]:
home = str(Path.home())
path = os.path.join(home,'github', 'mario_curiosity.scene_agents', 'outputdata','scene_clips', 'sub-*','sub-*', 'ses-*', 'beh', 'variables', '*.json')
file_list_base = glob.glob(path)

columns = ( 'Subject', 'World', 'Level', 'Scene','LevelFullName', 'SceneFullName','ClipCode',
        'Learning_Phase', 'player_x_posHi', 'player_x_posLo', 'player_y_pos','json_path')

# to add 'Phase', 'Phase_stage'

df_var_im = pd.DataFrame(columns=columns)

df_var_im['json_path'] = file_list_base
df_var_im['Subject'] = df_var_im['json_path'].str.extract(r'(sub-\d+)_ses')
df_var_im[['World', 'Level', 'Scene', 'ClipCode']] = df_var_im['json_path'].str.extract(r'level-w(\d+)l(\d+)_scene-(\d+)_clip-(\d+).json')
df_var_im['LevelFullName'] = 'w' + df_var_im['World'] + 'l' + df_var_im['Level']
df_var_im['SceneFullName'] = df_var_im['World']+'-'+df_var_im['Level']+'-'+df_var_im['Scene']
df_var_im['Learning_Phase'] = df_var_im['json_path'].str.extract(r'_(epoch\\=\d+-step\\=\d+)')


player_x_hi = []
player_x_lo = []
player_y = []

for path in df_var_hum['json_path']:
    with open(path, "r") as f:
        data = json.load(f)

    player_x_hi.append(data["player_x_posHi"])
    player_x_lo.append(data["player_x_posLo"])
    player_y.append(data["player_y_pos"])

df_var_im['player_x_posHi'] = player_x_hi
df_var_im['player_x_posLo'] = player_x_lo
df_var_im['player_y_pos'] = player_y

df_var_im.to_parquet("../sourcedata/df_variables_im.parquet", index=False)


AttributeError: Can only use .str accessor with string values!

In [ ]:
df_tot = pd.concat([df_var_hum, df_ppo_var], ignore_index=True)
df_tot.to_parquet("../sourcedata/df_variables.parquet", index=False)